In [ ]:
import pandas as pd
import numpy as np



In [ ]:

df=pd.read_csv('/content/drive/MyDrive/Maternal Health Risk Data Set.csv')

In [ ]:
df

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,25,130,80,15.0,98.0,86,high risk
1,35,140,90,13.0,98.0,70,high risk
2,29,90,70,8.0,100.0,80,high risk
3,30,140,85,7.0,98.0,70,high risk
4,35,120,60,6.1,98.0,76,low risk
...,...,...,...,...,...,...,...
1009,22,120,60,15.0,98.0,80,high risk
1010,55,120,90,18.0,98.0,60,high risk
1011,35,85,60,19.0,98.0,86,high risk
1012,43,120,90,18.0,98.0,70,high risk


In [ ]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1014 entries, 0 to 1013
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Age          1014 non-null   int64  
 1   SystolicBP   1014 non-null   int64  
 2   DiastolicBP  1014 non-null   int64  
 3   BS           1014 non-null   float64
 4   BodyTemp     1014 non-null   float64
 5   HeartRate    1014 non-null   int64  
 6   RiskLevel    1014 non-null   object 
dtypes: float64(2), int64(4), object(1)
memory usage: 55.6+ KB


In [ ]:
df.describe()

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate
count,1014.000000,1014.000000,1014.000000,1014.000000,1014.000000,1014.000000
mean,29.871795,113.198225,76.460552,8.725986,98.665089,74.301775
std,13.474386,18.403913,13.885796,3.293532,1.371384,8.088702
min,10.000000,70.000000,49.000000,6.000000,98.000000,7.000000
25%,19.000000,100.000000,65.000000,6.900000,98.000000,70.000000
50%,26.000000,120.000000,80.000000,7.500000,98.000000,76.000000
75%,39.000000,120.000000,90.000000,8.000000,98.000000,80.000000
max,70.000000,160.000000,100.000000,19.000000,103.000000,90.000000


In [ ]:
pip install scikit-learn

In [ ]:
import sklearn

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, classification_report

In [ ]:
X=df.drop('RiskLevel',axis=1)
y=df['RiskLevel']

In [ ]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Assudming you have your dataset loaded into `data` and your target variable is `RiskLevel`
data=df
# Step 1: Encode the categorical 'RiskLevel' column
label_encoder = LabelEncoder()
data['RiskLevel'] = label_encoder.fit_transform(data['RiskLevel'])

# Step 2: Split your dataset into features and target
X = data.drop(columns=['RiskLevel'])  # Features (excluding RiskLevel column)
y = data['RiskLevel']  # Target variable

# Step 3: Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Define and train the XGBoost model
model = xgb.XGBClassifier(
    n_estimators=1000,            # Number of trees
    learning_rate=0.05,           # Step size shrinking
    max_depth=8,                  # Maximum depth of a tree
    objective='multi:softmax',    # For multiclass classification
    num_class=len(label_encoder.classes_),  # Number of unique classes
    scale_pos_weight=[1.0, 1.0, 2.5],  # Class weights
    verbosity=1                   # Verbosity level
)

# Step 5: Train the model
model.fit(X_train, y_train)

# Step 6: Make predictions
y_pred = model.predict(X_test)

# Step 7: Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred))


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [04:51:37] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)


Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.87      0.86        47
           1       0.86      0.82      0.84        80
           2       0.82      0.84      0.83        76

    accuracy                           0.84       203
   macro avg       0.84      0.85      0.85       203
weighted avg       0.84      0.84      0.84       203



In [ ]:
import pickle

# Save the trained model using pickle
with open('risk_model.pkl', 'wb') as f:
    pickle.dump(model, f)


In [ ]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.6 MB/s eta 0:00:00


In [ ]:
import catboost

In [ ]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.05,
    depth=8,
    loss_function="MultiClass",
    class_weights=[1.0, 1.0, 2.5],
    verbose=100
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))

0:	learn: 1.0691464	total: 2.86ms	remaining: 2.86s
100:	learn: 0.4789389	total: 180ms	remaining: 1.61s
200:	learn: 0.3629612	total: 363ms	remaining: 1.44s
300:	learn: 0.2997326	total: 598ms	remaining: 1.39s
400:	learn: 0.2603789	total: 784ms	remaining: 1.17s
500:	learn: 0.2328206	total: 978ms	remaining: 974ms
600:	learn: 0.2148853	total: 1.16s	remaining: 771ms
700:	learn: 0.2021527	total: 1.34s	remaining: 573ms
800:	learn: 0.1923778	total: 1.55s	remaining: 385ms
900:	learn: 0.1846118	total: 1.73s	remaining: 191ms
999:	learn: 0.1783610	total: 1.92s	remaining: 0us
Classification Report:
              precision    recall  f1-score   support

   high risk       0.93      0.83      0.88        47
    low risk       0.91      0.72      0.81        80
    mid risk       0.73      0.93      0.82        76

    accuracy                           0.83       203
   macro avg       0.86      0.83      0.83       203
weighted avg       0.85      0.83      0.83       203



In [ ]:
import pandas as pd

custom_row = pd.DataFrame({
    "Age": [30],            # Age
    "SystolicBP": [120],    # Systolic Blood Pressure
    "DiastolicBP": [80],    # Diastolic Blood Pressure
    "BS": [5.5],            # Blood Sugar level
    "BodyTemp": [37.0],     # Body Temperature in Celsius
    "HeartRate": [72]       # Heart Rate in beats per minute
})

print(custom_row)



   Age  SystolicBP  DiastolicBP   BS  BodyTemp  HeartRate
0   30         120           80  5.5      37.0         72


In [ ]:
custom_row = pd.DataFrame({
    "Age": [45],
    "SystolicBP": [180],
    "DiastolicBP": [120],
    "BS": [8.0],
    "BodyTemp": [38.5],
    "HeartRate": [100]
})


In [ ]:
# Predict the class for the custom row
predicted_class = model.predict(custom_row)
print(f"Predicted Class: {predicted_class[0]}")


Predicted Class: ['high risk']


In [ ]:
model.save_model("risk_model.cbm")